In [1]:
import pandas as pd
import numpy as np  


import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
sol_phys_train = pd.read_csv('../data/interim/solar_phys_train.csv')
sol_phys_train = sol_phys_train.drop('Unnamed: 0', axis=1)
sol_phys_train.head()

,Radiation,Temperature,Pressure,Humidity,WindDirection(Degrees),Speed
0,634.99,289.26,4060.99,41,14.96,3.02
1,1.27,275.93,4034.32,70,207.43,2.51
2,1.21,281.48,4064.99,33,168.20,2.51
3,1.67,285.37,4060.99,101,152.60,1.51
4,839.78,289.82,4062.32,36,291.95,3.52


In [3]:
sol_phys_train.describe()

,Radiation,Temperature,Pressure,Humidity,WindDirection(Degrees),Speed
count,24514.000000,24514.000000,24514.000000,24514.000000,24514.000000,24514.000000
mean,208.062687,283.763923,4056.016801,75.002284,143.129305,2.793646
std,316.133238,3.452542,7.299362,25.999994,82.939584,1.556280
min,1.130000,274.260000,4024.990000,11.000000,0.090000,0.000000
25%,1.230000,280.930000,4052.990000,55.000000,81.872500,1.510000
50%,2.720000,283.150000,4056.990000,85.000000,147.585000,2.510000
75%,359.927500,285.930000,4060.990000,97.000000,179.217500,3.520000
max,1601.260000,294.820000,4074.320000,103.000000,359.950000,17.600000


In [4]:
def fix_angle(a):
    return round(a/360, 4)

In [5]:
sol_phys_train['WindDirection(Degrees)'] = sol_phys_train['WindDirection(Degrees)'].apply(fix_angle)

In [6]:
sol_phys_train.describe()

,Radiation,Temperature,Pressure,Humidity,WindDirection(Degrees),Speed
count,24514.000000,24514.000000,24514.000000,24514.000000,24514.000000,24514.000000
mean,208.062687,283.763923,4056.016801,75.002284,0.397581,2.793646
std,316.133238,3.452542,7.299362,25.999994,0.230388,1.556280
min,1.130000,274.260000,4024.990000,11.000000,0.000300,0.000000
25%,1.230000,280.930000,4052.990000,55.000000,0.227400,1.510000
50%,2.720000,283.150000,4056.990000,85.000000,0.409950,2.510000
75%,359.927500,285.930000,4060.990000,97.000000,0.497800,3.520000
max,1601.260000,294.820000,4074.320000,103.000000,0.999900,17.600000


In [7]:
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVR
import sklearn

In [8]:
from sklearn.model_selection import train_test_split
import optuna

In [9]:

X = sol_phys_train[[c for c in sol_phys_train if c != 'Radiation']]
y = sol_phys_train[['Radiation']]


In [10]:
from sklearn.preprocessing import MinMaxScaler

In [11]:
sc_X = MinMaxScaler()
sc_y = MinMaxScaler()
X = sc_X.fit_transform(X)
y = sc_y.fit_transform(y)

In [12]:
X_train, X_val, y_train, y_val = sklearn.model_selection.train_test_split(X, y,random_state=0, test_size=0.30)

In [13]:
def objective(trial):
# Invoke suggest methods of a Trial object to generate hyperparameters.

    svr_c = trial.suggest_loguniform('svr_c', 1e-10, 1e10)
    epsilon = trial.suggest_loguniform('epsilon', 1e-1, 1e1)
        

    #X = sol_phys_train[[c for c in sol_phys_train if c != 'Radiation']]
    #y = sol_phys_train[['Radiation']]
    #X_train, X_val, y_train, y_val = sklearn.model_selection.train_test_split(X, y,random_state=0)
    
    
    model = SVR(C=svr_c, epsilon=epsilon, gamma='auto')
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_val)
    error = sklearn.metrics.mean_squared_error(y_val, y_pred)
    
    
    return error # An objective value linked with the Trial object.

In [14]:
#study = optuna.create_study()
#study.optimize(objective, n_trials=100)
#study.best_params

In [15]:
# 8.733883088819892, 'epsilon': 0.12896899898357644
        
#Best is trial#2 with value: 0.039697788902204184.

In [16]:
svr_c = 8.733883088819892
epsilon = 0.12896899898357644

model = SVR(kernel='rbf', C=svr_c, epsilon=epsilon, gamma='auto')
model.fit(X_train, y_train)

/home/miguel/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:744: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



SVR(C=8.733883088819892, cache_size=200, coef0=0.0, degree=3,
    epsilon=0.12896899898357644, gamma='auto', kernel='rbf', max_iter=-1,
    shrinking=True, tol=0.001, verbose=False)

In [17]:
predict = model.predict(X_val)

In [18]:

np.sqrt(mean_squared_error(predict, y_val))

0.12199687209548707

In [19]:
sol_phys_test = pd.read_csv('../data/interim/solar_phys_test.csv')
sol_phys_test = sol_phys_test.drop('Unnamed: 0', axis=1)
sol_phys_test.head()

,id,Temperature,Pressure,Humidity,WindDirection(Degrees),Speed
0,0,288.15,4062.32,44,312.67,1.51
1,1,288.15,4063.65,83,38.01,3.02
2,2,281.48,4051.66,78,213.62,2.51
3,3,280.37,4052.99,98,176.63,2.01
4,4,280.37,4052.99,34,175.89,3.02


In [20]:
sol_phys_test['WindDirection(Degrees)'] = sol_phys_test['WindDirection(Degrees)'].apply(fix_angle)

In [21]:
X_t = sol_phys_test.drop('id',axis=1)

In [22]:

sc_X_t = MinMaxScaler()
X_t = sc_X_t.fit_transform(X_t)


In [23]:
predic_testf = model.predict(X_t)
submission2 = pd.DataFrame()
submission2['id'] = sol_phys_test['id']
submission2['Radiation'] = sc_y.inverse_transform(predic_testf.reshape(-1,1))

In [24]:
submission2.describe()

,id,Radiation
count,8172.000000,8172.000000
mean,4085.500000,231.227625
std,2359.197533,228.419410
min,0.000000,-254.626226
25%,2042.750000,96.818944
50%,4085.500000,158.847086
75%,6128.250000,317.154579
max,8171.000000,1208.689847


In [25]:
submission2['Radiation'] = submission2['Radiation'].apply(lambda x: x if x >= 0 else 0)
submission2.describe()

,id,Radiation
count,8172.000000,8172.000000
mean,4085.500000,237.533243
std,2359.197533,220.272418
min,0.000000,0.000000
25%,2042.750000,96.818944
50%,4085.500000,158.847086
75%,6128.250000,317.154579
max,8171.000000,1208.689847


In [26]:
submission2.to_csv('../data/processed/submission2.csv')

In [28]:
sol_phys_train.to_csv('../data/interim/solar_phys_train.csv')

In [30]:
sol_phys_test.to_csv('../data/interim/solar_phys_test.csv')